In [1]:
import pickle as pickle
import os
import pandas as pd
import torch

In [34]:
def load_data(dataset_dir):
  """ csv 파일을 경로에 맡게 불러 옵니다. """
  pd_dataset = pd.read_csv(dataset_dir)
  dataset = preprocessing_dataset(pd_dataset)
  
  return dataset

def preprocessing_dataset(dataset):
  """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
  subject_entity = []
  object_entity = []
  for i,j in zip(dataset['subject_entity'], dataset['object_entity']):
    i2 = i[1:-1].split(',')[0].split(':')[1]
    i=eval(i)['word']
    if len(i2[2:-1]) != len(i):
      print(i,i2)
    j2 = j[1:-1].split(',')[0].split(':')[1]
    j=eval(j)['word']
    if len(j2[2:-1]) != len(j):
      print(j,j2)


    subject_entity.append(i)
    object_entity.append(j)
  # print(subject_entity)
  out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],'subject_entity':subject_entity,'object_entity':object_entity,'label':dataset['label'],})
  # print(out_dataset)
  return out_dataset


In [37]:
train_dataset = load_data("../data/dataset/train/train_sample.csv")
train_dataset.head()

37,000명  '37
군인, 탐험가  '군인
제1, 3야전군  '제1
22,528.74 위안  '22
20,000명  '20
GitHub, Inc.  'GitHub
93,000명  '93
USANA Health Sciences, Inc.  'USANA Health Sciences
성삼빙, 성삼고, 성삼성  '성삼빙
56,000명  '56
BJ, 유튜버  'BJ
12,000명  '12
2,000여 명  '2
8,000명  '8
60,000명  '60
영화감독, 각본가  '영화감독
1억 8,000만 엔  '1억 8
임해군, 광해군  '임해군
1,041병상  '1
12,000명  '12
78,156명  '78
작곡가, 지휘자  '작곡가
모델, 배우, 영화 프로듀서  '모델
배우, 방송인  '배우
17,000명  '17
30, 000명  '30
스카, 록스테디, 레게  '스카
영화감독, 소설가  '영화감독
공격형 미드필더, 공격수  '공격형 미드필더
영화 감독, 영화 제작자, 배우, 각본가  '영화 감독
600,000명  '600
15,000명  '15
4,000 달러  '4
75,000명  '75
94,000명  '94
배우, 각본가, 연출가  '배우
영어 교사, 유튜버  '영어 교사
5,000명  '5
2,000명  '2
200,000명  '200
뮤지션, 작사가, 작곡가  '뮤지션
배우, 모델  '배우
10,000명  '10
41,000명  '41
님아, 그 강을 건너지 마오  '님아
1,161명  '1
8,000명  '8
피아노 연주자, 지휘자  '피아노 연주자
배우, 영화 제작자  '배우
152,000  '152
5,000만원  '5
2,000만불  '2
6,000  '6
The SSE Arena, Wembley  'The SSE Arena
가수, 배우  '가수
1,000  '1
가수, 작사가, 작곡가  '가수
Brian Elwin Haner, Jr  'Brian Elwin Haner
시인, 싱어송라이터  '시인
80,000  '80
작가, 시인, 언론인,

,id,sentence,subject_entity,object_entity,label
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,비틀즈,조지 해리슨,no_relation
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,민주평화당,대안신당,no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,광주FC,한국프로축구연맹,org:member_of
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,아성다이소,박정부,org:top_members/employees
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,요미우리 자이언츠,1967,no_relation
